### Elementy sztucznej inteligencji
# Projekt: Binarne Drzewa Decyzyjne
autorzy:
1. Arkadiusz Florek
2. Maciej Komosa
3. Albert Pieniądz
4. Jakub Zięba

Importowanie bibliotek, funkcji i obiektów

In [32]:
import numpy as np, pandas as pd
from math import log2
from graphviz import Digraph

Obliczanie entropii na podstawie ilości wystąpienia konkluzji w danych

In [33]:
def _entropia(dane):
    '''
    słownik:
    klucz - konkluzja
    wartość - liczba wystąpień konkluzji
    '''
    slownik = {}
    for _ in dane:
        atr = _[-1]
        if atr not in slownik:
            slownik[atr] = 0
        slownik[atr] += 1

    '''
    entropia:
    suma p * log2(p)
    '''
    entropia = 0.0
    for _ in slownik.values():
        p = _ / len(dane)
        entropia -= p * log2(p)

    return entropia

Obliczanie zysku informacji na podstawie informacji zawartej w danych oraz entropii dla danej przesłanki

In [34]:
def _zysk_informacji(dane, przeslanka):
        '''
        slownik:
        klucz - wartość przesłanki
        wartość - lista wierszy spełniających przesłankę
        '''
        slownik = {}
        for _ in dane:
            wartosc = _[przeslanka]
            if wartosc not in slownik:
                slownik[wartosc] = []
            slownik[wartosc].append(_)
    
        '''
        obliczanie entropii przed i po podzieleniu
        '''
        iloscWierszy = len(dane)
        entropie = []
        for wartosciPrzeslanki in slownik.values():
            iloscWartosciPrzeslanki = len(wartosciPrzeslanki)
            entropiaPrzeslanki = _entropia(wartosciPrzeslanki)
            entropie.append((iloscWartosciPrzeslanki / iloscWierszy) * entropiaPrzeslanki)
        entropiaPrzeslanki = sum(entropie)
    
        '''
        zysk informacji
        '''
        zysk = _entropia(dane) - entropiaPrzeslanki
    
        return zysk

Obliczanie przesłanki o największym zysku

In [35]:
def _przeslanka_o_najwiekszym_zysku(dane, nazwyPrzeslanek, przeslankiIndeksy):

        '''
        oblicza dla wszystkich przesłanek zyski i wybiera tę najlepszą
        '''
        
        entropiePrzeslanek = [_zysk_informacji(dane, przeslanka) for przeslanka in przeslankiIndeksy]
        indeksPrzeslankaONajwiekszymZysku = przeslankiIndeksy[entropiePrzeslanek.index(max(entropiePrzeslanek))]

        if max(entropiePrzeslanek) == 0:
                return None, None
        
        return nazwyPrzeslanek[indeksPrzeslankaONajwiekszymZysku], indeksPrzeslankaONajwiekszymZysku

Algorytm id3 - budowa binarnego drzewa decyzyjnego

In [36]:
def _id3(dane, atrybuty):
        '''
        inicjalizuje drzewo, czyli listę zbudowaną z kolejnych węzłów
        i połączeń pomiędzy nimi
        '''
        drzewo = [] 
        '''
        slownik:
        klucz - konkluzja
        wartość - liczba wierszy dla danej konkluzji
        '''
        (klucze, czestosci) = np.unique(dane[:, -1], return_counts=True)
        slownik = dict(zip(klucze, czestosci))
        '''
        dla każdej konkluzji sprawdza, czy jest jedyną konkluzją w zbiorze.
        jeśli tak, zwraca konkluzję jako węzeł liściowy
        '''
        for wartosc in slownik:
            if slownik[wartosc] == len(dane):
                return wartosc
        '''
        jeśli nie ma atrybutów, zwraca konkluzję o największej liczbie wystąpień
        '''
        if len(atrybuty) == 1:
               return max(slownik, key=slownik.get)
        '''
        oblicza przeslankę o największym zysku, dodaje ją do drzewa i usuwa z listy atrybutów
        '''
        przeslanka, indeksPrzeslanka = _przeslanka_o_najwiekszym_zysku(dane, atrybuty, range(len(atrybuty)))
        if przeslanka is None:
                return max(slownik, key=slownik.get)
        drzewo.append(przeslanka)
        atrybuty = np.delete(atrybuty, indeksPrzeslanka)

        '''
        dla każdej wartości przesłanki tworzy poddrzewo, a następnie
        dodaje je do drzewa
        '''
        wartosciPrzeslanki = np.unique(dane[:, indeksPrzeslanka])
        
        poddrzewa = []

        for wartosc in wartosciPrzeslanki:
                przeslanka, indeksPrzeslanka = _przeslanka_o_najwiekszym_zysku(dane, atrybuty, range(len(atrybuty)))
                poddrzewo = [wartosc, _id3(dane[dane[:, indeksPrzeslanka] == wartosc], atrybuty)]
                poddrzewa.append(poddrzewo)
                drzewo.append(poddrzewo)

        if poddrzewa[0][1] == poddrzewa[1][1]:
                drzewo = poddrzewa[0][1]

        return drzewo

Reprezentacja binarnego drzewa decyzyjnego za pomocą pakietu Graphviz

In [37]:
def _rysuj_drzewo(drzewo, graf=None, i=0, s = ""):

        if i == 0:
                graf = Digraph(comment='Drzewo decyzyjne')
                graf.node(s + str(i), drzewo[0])

        #galezie
        for j in range(1, len(drzewo)):

                decyzja = drzewo[j][0] == "0" and "nie" or "tak"
                graf.edge(s + str(i), s + str(i) + str(j), label=decyzja)

                if isinstance(drzewo[j][1], str):
                        graf.node(s + str(i) + str(j), drzewo[j][1])                        
                else:
                        graf.node(s + str(i) + str(j), drzewo[j][1][0])
                        _rysuj_drzewo(drzewo[j][1], graf, j, s + str(i))
                
        return graf


Prezentacja drzewa zbudowanego za pomocą tabeli przygotowanej ręcznie

In [38]:
dane = pd.read_csv("sport_binarnie.csv", header=None).to_numpy()
_rysuj_drzewo(_id3(dane[1:, :], dane[0, :-1])).render('drzewo', view=True)

'drzewo.pdf'

### suplement - drzewo niebinarne (bez przycinania)
modyfikacje funkcji

In [39]:
def _przeslanka_o_najwiekszym_zysku_2(dane, nazwy_przeslanek, przeslanki_indeksy):

        entropie_przeslanek = [_zysk_informacji(dane, przeslanka) for przeslanka in przeslanki_indeksy]
        indeks_przeslanka_o_najwiekszym_zysku = przeslanki_indeksy[entropie_przeslanek.index(max(entropie_przeslanek))]

        return nazwy_przeslanek[indeks_przeslanka_o_najwiekszym_zysku], indeks_przeslanka_o_najwiekszym_zysku

def _id3_2(dane, atrybuty):
        
        drzewo = [] 
        (unique, counts) = np.unique(dane[:, -1], return_counts=True)
        slownik = dict(zip(unique, counts))
        for wartosc in slownik:
            if slownik[wartosc] == len(dane):
                drzewo.append(wartosc)
                return drzewo
        if len(atrybuty) == 0:
                drzewo.append(max(slownik, key=slownik.get))
                return drzewo
        przeslanka, indeks_przeslanka = _przeslanka_o_najwiekszym_zysku_2(dane, atrybuty, range(len(atrybuty)))
        drzewo.append(przeslanka)
        atrybuty = np.delete(atrybuty, indeks_przeslanka)
        for wartosc in np.unique(dane[:, indeks_przeslanka]):
                podzbior = dane[dane[:, indeks_przeslanka] == wartosc]
                podzbior = np.delete(podzbior, indeks_przeslanka, axis=1)
                drzewo.append([wartosc, _id3_2(podzbior, atrybuty)])

        return drzewo

def _rysuj_drzewo_2(drzewo, graf=None, i=0, s = ""):
        if graf is None:
                graf = Digraph()
        if i == 0:
                graf.node(s + str(i), drzewo[0])
        for j in range(1, len(drzewo)):
                graf.node(s + str(i) + str(j), drzewo[j][1][0])
                graf.edge(s + str(i), s + str(i) + str(j), label=drzewo[j][0])
                _rysuj_drzewo(drzewo[j][1], graf, j, s + str(i))
                
        return graf

importowanie danych i rysowanie drzewa

In [40]:
dane = pd.read_csv("sport.csv", header=None).to_numpy()
_rysuj_drzewo_2(_id3_2(dane[1:, :], dane[0, :-1])).render('drzewo niebinarne', view=True)

'drzewo niebinarne.pdf'